# 8fit Data analyst case solution

I use this Jupyter Notebook for creating my final report and exploring the data.

In [1]:
import pandas as pd 
import numpy as np

### Loading the Data

In [2]:
subscriptions = pd.read_csv("subscriptions.csv")
spendings = pd.read_csv("spendings.csv")

### Merging subscription and spendings datasets

In [4]:
merged_df = pd.merge(
    subscriptions,
    spendings,
    how="left",
    left_on=[
        "signup_country_code",
        "marketing_channel_id",
        "signup_platform",
        "subscription_date",
    ],
    right_on=["country_code", "marketing_channel_id", "platform", "report_date"],
)
df = merged_df.dropna()

## Question 1.
### Where to invest? ios or android?

### Solution:
One of the most important KPIs in marketing analysis is CAC,Cost of customer acquisition.
Below Visualizations help us to have a more clear idea on both platforms' perfromance. 

<img src="Platform.png">

As it is seen above we have spent more in ios and consequently got more revenue from ios. 
##### In terms of CAC android is doing better. CAC for android is around 7% lower than ios.
### Therefore, I suggest to invest more in android


## Question 2. 
### Is there any specific marketing channels we should be aiming for to get better results? 
### I'm currently thinking that the channel with ID 4 looks promising. What are your thoughts? 

### Solution:
As it is understood from the data, there are some paid and unpaid channels available. So I would answer to this question for paid and unpaid categories separately.
I have also defined a new KPI,Marketing Spendings/Revenue, which could be an indicator of marketing ROI in each channel. The lowest the Marketing Spendings/Revenue, the better.

#### - Unpaid Channels:
    Among unpaid channels, Channel with ID 1 is outperforming both paid and unpaid ones by far. In my opinion, it is the most successful one. And if our startegy is not to necessarily invest in paid channels, channel 1 would be the best to invest in.

#### - Paid Channels:
    It seems that Channel with ID 2 is the one doing better than other paid channels; as it has the highest revenue share and lowest CAC among paid channels.
    Apart from channel 10, channel 2 has also the lowest Marketing_Spending/Revenue among other paid channels.
    I would not choose channel 10 because of its low revenue share.

<img src="Channels.png">

## Question 3.
#### Looking further into the data at hand it looks like UK (United Kingdom) would be a good investment. Do you agree? 

### Solution:

Having aggregated the data on the countries level, I sorted them based on net_revenue, spendings, CAC respectively. 

In [7]:
Countries=df.groupby('signup_country_code')['spendings','net_revenue','subscription_count'].sum()
Countries['CAC']=Countries['spendings']/Countries['subscription_count']
Countries['MarketingCost/Revenue']=Countries['spendings']/Countries['net_revenue']
Countries.sort_values(['net_revenue','spendings','CAC'],ascending=False)[0:15]

,spendings,net_revenue,subscription_count,CAC,MarketingCost/Revenue
signup_country_code,,,,,
US,1.101293e+06,474061.606983,38418,28.666061,2.323100
CA,2.176953e+05,91924.064864,7702,28.264778,2.368208
CH,1.070114e+05,69317.169287,3973,26.934656,1.543793
AU,1.104261e+05,64562.365066,4630,23.850130,1.710379
FR,7.299754e+04,56318.861900,4313,16.925003,1.296147
ES,1.172919e+05,52255.592894,5299,22.134715,2.244580
GB,6.887108e+04,43444.365635,4727,14.569723,1.585271
MX,8.220753e+04,41640.531544,4817,17.066126,1.974219
AR,3.425828e+04,37823.633723,2533,13.524785,0.905737


In my opinion among the countries with highest revenue share AR,Argentina, seems to be more intersting as it shows a low CAC and MarketingCost/Revenue, which means that it would be a profitable market with a high marketing ROI.

UK which is available as GB in this dataset seems also a nice choice but its indicators are still worse than AR.


##### Overal, if we aim to invest in Europe UK would be a promising market, but globally AR outperforms it. Therefore, if 8fit does not have any strategic prefrences in order to grow first in EU, it should go for AR otherwise UK would be the best.

# SQL Question

I did not use the docker setup, therefore; I loaded the two datasets into two tables, subscriptions and spendings, in my local postgresql and did the query.

### 1. How much did we spent per channel in December?

In [ ]:
SELECT marketing_channel_id,SUM(spendings) AS total_spendings from spendings
WHERE EXTRACT(MONTH FROM report_date)=12
GROUP BY marketing_channel_id
ORDER BY total_spendings DESC;

<img src='Q1.PNG'>

### 2. What is the average cost of acquisition for a subscription per country?

In [ ]:
SELECT signup_country_code as country, SUM(spendings)/SUM(subscription_count) AS CAC
FROM subscriptions S INNER JOIN spendings M ON
S.signup_country_code=M.country_code 
AND S.marketing_channel_id=M.marketing_channel_id
AND S.subscription_date=M.report_date 
GROUP BY signup_country_code 
ORDER BY CAC DESC;

<img src='Q2.PNG'>

### 3. What is our average revenue and spending per day of the week (Monday, Tuesday...)?


In [ ]:
SELECT to_char(subscription_date,'Day') as DayOfWeek, AVG(net_revenue) as avg_revenue, AVG(spendings) as avg_spendings
FROM subscriptions S INNER JOIN spendings M ON
S.signup_country_code=M.country_code
AND S.marketing_channel_id=M.marketing_channel_id
AND S.subscription_date=M.report_date
GROUP BY to_char(subscription_date,'Day');

<img src='Q3.PNG'>